<a href="https://colab.research.google.com/github/ammobam/Display_SensorData/blob/main/step_1_%EC%A0%84%EC%B2%98%EB%A6%AC_%EC%88%98%ED%96%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 머신러닝을 이용한 중요 피처 찾기
- 데이터 : 디스플레이 공정별 센서 측정값. (.csv)

## 할 것
- csv 파일을 넣으면 머신러닝까지 자동으로 수행되도록 함

## 패키지 임포트

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
#help(pd.read_csv)

## 데이터 불러오기
- 데이터 불러오기
- 원본데이터 보존을 위해 copy 데이터 만들어서 사용하기
- (선택) 생산라인 L, R 구분

In [14]:
# 데이터 불러오기
# 첫번째 열을 인덱스로 사용함
fact_data = pd.read_csv('/content/drive/MyDrive/나무플래닛/1. 데이터_디스플레이/factory_glass_2016.zip', encoding='cp949', index_col=0)
    # encoding : 파일이 안 열리면 인코딩 옵션을 확인하자. utf-8, cp949등으로 설정해봄.
    # index_col : 인덱스로 사용할 columns의 인덱스를 설정

In [15]:
# 데이터 확인
fact_data.info()
fact_data.head()
    # 840 컬럼 x 8145개 행이 있음
    # 데이터 타입은 float과 int형으로 모두 숫자 데이터. 연산 가능함.

<class 'pandas.core.frame.DataFrame'>
Index: 8145 entries, 2016-01-01 07 to 2016-12-31 23
Columns: 840 entries, LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101 to R.Vac
dtypes: float64(834), int64(6)
memory usage: 52.3+ MB


,LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101,LIFT.OUT.ROL.MTR.M2.SPD.1WSI40102,LIFT.OUT.ROL.MTR.M3.SPD.1WSI40103,LEHR.DRV.MTR..M4.1.SPD.1WSI40104,LEHR.DRV.MTR..M4.2.SPD.1WSI40105,LEHR.DRV.MTR..M5.1.SPD.1WSI40106,LEHR.DRV.MTR..M5.2.SPD.1WSI40107,LEHR.DRV.MTR..M6.1.SPD.1WSI40108,LEHR.DRV.MTR..M6.2.SPD.1WSI40109,LEHR.DRV.MTR..M7.1.SPD.1WSI40110,LEHR.DRV.MTR..M7.2.SPD.1WSI40111,LEHR.DRV.MTR..M8.1.SPD.1WSI40112,LEHR.DRV.MTR..M8.2.SPD.1WSI40113,LEHR.DRV.MTR..M9.1.SPD.1WSI40114,LEHR.DRV.MTR..M9.2.SPD.1WSI40115,DRIVE.MOTOR.M10.1.SPEED.1WSI40116,LEHR.DRV.MTR..M10.2.SPD.1WSI40117,LEHR.DRV.MTR..M11.1.SPD.1WSI40118,DRIVE.MOTOR.M11.2.SPEED.1WSI40119,LEHR.DRV.MTR..M12.1.SPD.1WSI40120,LEHR.DRV.MTR..M12.2.SPD.1WSI40121,LEHR.DRV.MTR..M13.1.SPD.1WSI40122,LEHR.DRV.MTR..M13.2.SPD.1WSI40123,LEHR.DRV.MTR..M14.1.SPD.1WSI40124,LEHR.DRV.MTR..M14.2.SPD.1WSI40125,LEHR.DRV.MTR..M15.1.SPD.1WSI40126,LEHR.DRV.MTR..M15.2.SPD.1WSI40127,LEHR.DRV.MTR..M16.1.SPD.1WSI40128,DRIVE.MOTOR.M16.2.SPEED.1WSI40129,LEHR.DRV.MTR..M17.1.SPD.1WSI40130,LEHR.DRV.MTR..M17.2.SPD.1WSI40131,LEHR.DRV.MTR..M18.1.SPD.1WSI40132,LEHR.DRV.MTR..M18.2.SPD.1WSI40133,DRIVE.MOTOR.M19.1.SPEED.1WSI40134,LEHR.DRV.MTR..M19.2.SPD.1WSI40135,X1WOS45101_PV,X1WOS45102_PV,X1WOS45103_PV,X1WOS45104_PV,X1WOS45105_PV,...,S_C1_C2_L_L,S_C1_C2_L_R,S_C2_C3_U_L,S_C2_C3_U_R,S_C2_C3_L_L,S_C2_C3_L_R,S_C3_C4_U_L,S_C3_C4_U_R,S_C3_C4_L_L,S_C3_C4_L_R,S_AB1_AB11_L,S_AB1_AB11_R,S_AB11_AB7_L,S_AB11_AB7_R,S_AB7_C4_L,S_AB7_C4_R,S_C4_RET_L,S_C4_RET_R,S_RET_F_L,S_RET_F_R,STD_U_L,STD_U_R,STD_L_L,STD_L_R,STD_GLS_L,STD_GLS_R,투입.LEFT.,투입.RIGHT.,ROLLDOWN.폐기.LEFT.,진공패드.폐기.LEFT.,ROLLDOWN.폐기.RIGHT.,진공패드.폐기.RIGHT.,폐기율...,ROLLDOWN.진공패드.폐기율..LEFT.,ROLLDOWN.진공패드.폐기율..RIGHT.,ROLLDOWN.진공패드.폐기율..전체.,L.RD,L.Vac,R.RD,R.Vac
dat...date.name.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-01 07,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.8189,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.73,-1.315444,-1.620631,-1.517655,-1.804912,...,-0.195736,0.706941,1.477203,0.410847,0.491620,0.942180,2.776941,2.492437,2.060406,2.612283,1.726404,1.461145,-1.192642,-1.134471,2.599429,2.690433,-0.365598,-0.659319,-1.963403,-1.621407,0.523950,1.951349,2.248106,2.767676,3.012706,3.069007,80,74,2,5,8,3,11.69,8.75,14.86,11.69,0.025,0.063,0.108,0.041
2016-01-01 08,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.8189,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.73,-1.315444,-1.620631,-1.517655,-1.804912,...,-0.500742,0.822657,1.495204,0.397591,0.587871,0.938186,2.888919,2.573187,2.161149,2.664726,1.682081,1.414518,-1.109125,-1.042269,2.588703,2.654307,-0.348979,-0.560103,-1.978936,-1.810022,0.566521,2.010125,2.364418,2.842630,3.022998,3.069274,92,75,0,2,10,0,7.19,2.17,13.33,7.19,0.000,0.022,0.133,0.000
2016-01-01 09,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.8189,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.73,-1.315444,-1.620631,-1.517655,-1.804912,...,-0.164556,0.962210,1.552127,0.460702,0.397216,0.843443,2.868644,2.486441,2.183298,2.638130,1.632955,1.359022,-0.986580,-0.928583,2.550175,2.606185,-0.456071,-0.644073,-1.878611,-1.742395,0.545936,2.002820,2.397034,2.817728,2.936967,2.969952,64,66,1,4,0,0,3.85,7.81,0.00,3.85,0.016,0.063,0.000,0.000
2016-01-01 10,-1.772682,-1.713219,-

### ** 데이터 복사에 관하여
- 복사를 하는 이유
    - 데이터를 복사하여 작업하면 원본 데이터의 유실을 예방할 수 있음.
    - 같은 데이터를 새로운 객체에 두 번 할당하는 경우에 비해 memory를 아낄 수 있음
- 복사의 종류
    -  a = 10, b = a로 복사하면 참조만 복사. 같은 데이터
    - [mutable문제 발생] 어느 한 쪽을 수정하면 다른 쪽도 변경됨
    - 얕은 복사(.copy())
        - 겉 객체를 복사하여 다른 객체 할당함
        - 원소가 다시 리스트 등의 객체인 경우, 그 원소객체의 참조만 복사되어 mutable 문제가 발생함
    - 깊은 복사(.deepcopy())
        - 겉 객체, 원소 객체가 2중인 경우 그 객체까지 복사 (예시 : 리스트 안에 다시 리스트)
        - mutable한 문제가 해결됨
- 더 알아보기 : 2중 배열 만들어서 copy, deepcopy 비교하기
    - 참고링크 : https://maeng-test-blog.tistory.com/98

In [16]:
# 원본데이터의 보존을 위해 얕은 복사를 수행함
# fact_data 데이터는 숫자형으로 얕은 복사만 수행해도 됨
fact_data_copy = fact_data.copy()

## 데이터 전처리
- 결측치 처리
- 분산 0인 피처 제거
- 상관관계 높은 피처 제거
- VIF > 30 이상의 피처 제거

### 컬럼 제외하기

#### 레이블 컬럼 제외하기

In [17]:
# 레이블 컬럼 제외하기
col_label = ['L.RD', 'L.Vac', 'R.RD', 'R.Vac']
col_use = []
for col in fact_data.columns:
    if col not in col_label:
       col_use.append(col)
print(len(col_use)) # 836개

# 데이터프레임 만들기
fact_data = fact_data[col_use]

836


In [18]:
# 레이블 컬럼에 혹시 결측치 있었나 확인
fact_data_copy[col_label].isnull().sum()
# 없음

L.RD     0
L.Vac    0
R.RD     0
R.Vac    0
dtype: int64

#### 아무리봐도 피처는 아닌 것 같은 컬럼 제거하기

In [19]:
fact_data.columns[-12:]

Index(['STD_GLS_L', 'STD_GLS_R', '투입.LEFT.', '투입.RIGHT.', 'ROLLDOWN.폐기.LEFT.',
       '진공패드.폐기.LEFT.', 'ROLLDOWN.폐기.RIGHT.', '진공패드.폐기.RIGHT.', '폐기율...',
       'ROLLDOWN.진공패드.폐기율..LEFT.', 'ROLLDOWN.진공패드.폐기율..RIGHT.',
       'ROLLDOWN.진공패드.폐기율..전체.'],
      dtype='object')

In [20]:
# 아무리봐도 피처는 아닌 것 같은 컬럼 제거하기
col_susang = fact_data.columns[-10:]

col_use = []
for col in fact_data.columns:
    if col not in col_susang:
       col_use.append(col)
print(len(col_use)) # 826개

# 데이터프레임 만들기
fact_data = fact_data[col_use]

826


#### Right 컬럼 제외하기

In [21]:
# Right 공정에 해당하는 컬럼
col_right = [   'DB.N2.SCREEN.FLOW.RIGHT.1FI49012_PV',
                 'DB.N2.SCREEN.FLOW.RIGHT.1FI49012_PV.1',
                 'Right.edge',
                 'TMP.TIN..BAY.1.RIGHT.1TI30202.PV',
                 'TMP.TIN..BAY.5.RIGHT.1TI30206.PV',
                 'TMP.TIN..BAY.7.RIGHT.1TI30208.PV',
                 'TMP.TIN..BAY.10.RIGHT.1TI30210.PV',
                 'TMP.GLASS..RIGHT.EXIT.1TIC30111.PV',
                 'X.10.BAY.RIGHT.PRESSURE.1CLBAY10RIGHT_CPV',
                 'X.1.BAY.RIGHT.PRESSURE.1CLBAY1RIGHT_CPV',
                 'X.7.BAY.RIGHT.PRESSURE.1CLBAY7RIGHT_CPV',
                 'RET.AMBIANT.RIGHT.TMP.1TI42603.PV',
                 'BATH.AMBIENT.3.BAY.RIGHT.1TI31002_PV',
                 'EXIT.LIP.PLATE.RIGHT.1TI30605.PV',
                 'E.L.DB.RIGHT.N2.FLOW.1FI33502.PV',
                 'DROSS.BOX.N2.BTM.HT1.R.1JI39006.PV',
                 'DROSS.BOX.N2.BTM.HT..R..1JI39010.PV',
                 'DROSSBOX.N2.SCN.R.1JI39012_PV',
                 'HOOD.N2.HEATER.R..1JI39008.PV',
                 'DDP.RIGHT.N2.H2.9..1FI33421.PV',
                 'SPOUT.RIGHT.UPSTREAM.1FI33516.PV',
                 'SPOUT.RIGHT.BOS.UPSTREAM.1FI33517.PV',
                 'SPOUT.RIGHT.BOTTOM.1FI33518.PV',
                 'SPOUT.RIGHT.TOP.1FI33519.PV',
                 'LOR.1.R.Axis',
                 'LOR.1.R.Hori',
                 'LOR.2.R.Axis',
                 'LOR.2.R.Hori',
                 'LOR.3.R.Axis',
                 'LOR.3.R.Hori',
                 'r_tin_bay_1_5',
                 'r_tin_bay_5_7',
                 'r_tin_bay_7_10',
                 'r_top_n.1_mean',
                 'r_top_n.2_mean',
                 'r_n.2_glass',
                 'D_AB1_R_UL',
                 'D_AB2_R_UL',
                 'D_AB3_R_UL',
                 'D_AB4_R_UL',
                 'D_AB5_R_UL',
                 'D_AB6_R_UL',
                 'D_AB7_R_UL',
                 'D_AB8_R_UL',
                 'D_AB9_R_UL',
                 'D_AB10_R_UL',
                 'D_AB11_R_UL',
                 'D_C1_R_UL',
                 'D_C2_R_UL',
                 'D_C3_R_UL',
                 'D_C4_R_UL',
                 'S_AB1_AB2_U_R',
                 'S_AB1_AB2_L_R',
                 'S_AB2_AB3_U_R',
                 'S_AB2_AB3_L_R',
                 'S_AB3_AB4_U_R',
                 'S_AB3_AB4_L_R',
                 'S_AB4_AB5_U_R',
                 'S_AB4_AB5_L_R',
                 'S_AB5_AB6_U_R',
                 'S_AB5_AB6_L_R',
                 'S_AB6_AB7_U_R',
                 'S_AB6_AB7_L_R',
                 'S_AB7_AB8_U_R',
                 'S_AB7_AB8_L_R',
                 'S_AB8_AB9_U_R',
                 'S_AB8_AB9_L_R',
                 'S_AB9_AB10_U_R',
                 'S_AB9_AB10_L_R',
                 'S_AB10_AB11_U_R',
                 'S_AB10_AB11_L_R',
                 'S_AB11_C1_U_R',
                 'S_AB11_C1_L_R',
                 'S_AB1_AB11_R',
                 'S_AB11_AB7_R',
                 'S_AB7_C4_R',
                 'S_C1_C2_U_R',
                 'S_C1_C2_L_R',
                 'S_C2_C3_U_R',
                 'S_C2_C3_L_R',
                 'S_C3_C4_U_R',
                 'S_C3_C4_L_R',
                 'S_C4_RET_R',
                 'S_RET_F_R',
                 'STD_U_R',
                 'STD_L_R',
                 'STD_GLS_R',
                 '투입.RIGHT.',
                 'ROLLDOWN.폐기.RIGHT.',
                 'ROLLDOWN.진공패드.폐기율..RIGHT.',
                 '진공패드.폐기.RIGHT.']
# 개수 확인
len(col_right) # 91개

91

In [22]:
# Right 컬럼 제외하기
col_use = []
for col in fact_data.columns:
    if col not in col_right:
       col_use.append(col)
print(len(col_use)) # 739개

# 데이터프레임 만들기
fact_data = fact_data[col_use]

739


### 결측치 처리

In [23]:
# 결측치 찾기
nan_data = fact_data.isnull().sum().sort_values(ascending=False)
print(nan_data)
## isnull() : 결측치를 찾아 True/False 리턴함.
    # boolean 데이터타입은 숫자로 변환하면 True -> 1, False -> 0
    # 여기서는 결측치가 있는 데이터는 1의 값을 갖게 됨
## sum() : 컬럼별로 값을 모두 더함
## sort_values() : 값을 크기별로 정렬함
    # ascending=False 옵션을 주면 내림차순으로 정렬함

LEHR.DRV.MTR..M19.2.CUR.1WII40135.PV    8145
ROOF.HEATING.Zone78.1JIC31178.PV        8145
DCS_Offset_RPM_M14.1WOS45114_PV         8145
UPPER.RET..Z218.TMP.1TIC42601.PV        8145
DCS_Offset_RPM_M13.1WOS45113_PV         8145
                                        ... 
ROOF.HEATING.Zone.9.1JIC31109.PV.1         0
ROOF.HEATING.Zone.8.1JIC31108.PV.1         0
ROOF.HEATING.Zone.7.1JIC31107.PV.1         0
ROOF.HEATING.Zone.6.1JIC31106.PV           0
LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101          0
Length: 739, dtype: int64


In [24]:
# 이때 결측치 개수 컬럼은 시리즈 타입이므로 슬라이싱 됨. nan 개수 상위 20개 정도 살펴보자.
print(type(nan_data))

# 조회 결과, 7개 컬럼은 전체 8145개의 행이 모두 결측치임. 데이터가 없으므로 삭제.
print(nan_data[:20])

<class 'pandas.core.series.Series'>
LEHR.DRV.MTR..M19.2.CUR.1WII40135.PV    8145
ROOF.HEATING.Zone78.1JIC31178.PV        8145
DCS_Offset_RPM_M14.1WOS45114_PV         8145
UPPER.RET..Z218.TMP.1TIC42601.PV        8145
DCS_Offset_RPM_M13.1WOS45113_PV         8145
ROOF.HEATING.Zone76.1JIC31176.PV        8145
ROOF.HEATING.Zone52.1JIC31152.PV        8145
N2.TO.EA1313A.1FI33503A_PV.1               1
LEHR.DRV.MTR..M9.2.SPD.1WSI40115.1         1
LEHR.DRV.MTR..M9.1.SPD.1WSI40114.1         1
LEHR.DRV.MTR..M8.2.SPD.1WSI40113.1         1
LEHR.DRV.MTR..M8.1.SPD.1WSI40112.1         1
LEHR.DRV.MTR..M7.2.SPD.1WSI40111.1         1
Ribbon.Speed                               1
LEHR.DRV.MTR..M7.1.SPD.1WSI40110.1         1
LEHR.DRV.MTR..M6.2.SPD.1WSI40109.1         1
LEHR.DRV.MTR..M6.1.SPD.1WSI40108.1         1
DRIVE.MOTOR.M10.1.SPEED.1WSI40116.1        1
LEHR.DRV.MTR..M5.2.SPD.1WSI40107.1         1
LEHR.DRV.MTR..M4.2.SPD.1WSI40105.1         1
dtype: int64


In [25]:
# 전체 행이 결측치인 컬럼을 찾고 삭제하기

# 1. 전체 행이 결측치인 컬럼을 찾기
print(nan_data[nan_data.values == fact_data.shape[0]].index.tolist())
    # nan_data에서 다음 조건에 해당하는 컬럼을 찾음
        # nan_data.values : nan_data를 데이터 배열로 만들어줌
        # fact_data.shape -> (행 개수, 열 개수)
            # fact_data.shape[0]는 fact_data 행의 개수임
    # .index : 그때의 컬럼이름(여기선 인덱스)을 추출함
    # .tolist() : 데이터를 리스트 타입으로 변환함

# 2. 해당 컬럼삭제하기
fact_data.drop(nan_data[nan_data.values == fact_data.shape[0]].index.tolist(), axis=1, inplace=True)
    # inplace=True : drop 수행한 결과를 fact_data에 반영할지 여부 설정

# 3. 확인 - 컬럼 수가 줄어들었나 확인 (840개 -> 833개)
fact_data.info()

['LEHR.DRV.MTR..M19.2.CUR.1WII40135.PV', 'ROOF.HEATING.Zone78.1JIC31178.PV', 'DCS_Offset_RPM_M14.1WOS45114_PV', 'UPPER.RET..Z218.TMP.1TIC42601.PV', 'DCS_Offset_RPM_M13.1WOS45113_PV', 'ROOF.HEATING.Zone76.1JIC31176.PV', 'ROOF.HEATING.Zone52.1JIC31152.PV']
<class 'pandas.core.frame.DataFrame'>
Index: 8145 entries, 2016-01-01 07 to 2016-12-31 23
Columns: 732 entries, LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101 to STD_GLS_L
dtypes: float64(732)
memory usage: 45.5+ MB


In [26]:
# 그 외 결측치 처리
# 남은 결측치는 ffill로 채워주자
# 이 데이터는 1시간마다 측정한 센서 데이터이므로 전체 컬럼에 대한 평균보다는 이전 행의 값으로 결측치를 처리하는 것이 적절함
fact_data.fillna(method = 'ffill', inplace=True)
    # inplace=True : fillna 수행한 결과를 fact_data에 반영할지 여부 설정

# 확인
fact_data.isnull().sum().sort_values()

LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101     0
ROOF.HEATING.Zone.4.1JIC31104.PV.1    0
ROOF.HEATING.Zone.5.1JIC31105.PV      0
ROOF.HEATING.Zone.6.1JIC31106.PV      0
ROOF.HEATING.Zone.7.1JIC31107.PV.1    0
                                     ..
LOWER.C3..Z193TMP.1TIC42454.PV        0
LOWER.C3..Z196.TMP.1TIC42457.PV       0
UPPER.C4..Z197.TMP.1TIC42501.PV       0
UPPER.C4..Z199.TMP.1TIC42503.PV       0
STD_GLS_L                             0
Length: 732, dtype: int64

### 분산 0인 데이터 제거
- 분산이 0인 데이터를 제거하는 이유?
    - 어떤 피처의 분산이 0이라는 것은 그 피처의 데이터가 모든 행에 대해 거의 변하지 않은 것을 의미함
    - 어떤 경우에도 같은 값을 내는 컬럼이 불량률에 영향을 주고 있다고 보기 어려움
- 수행 방법
    - 방법1 : sklearn의 VarianceThreshold 사용
    - 방법2 : var() 사용
- 결과
    - 컬럼수 : 831->818


In [27]:
# 방법1
#from sklearn.feature_selection import VarianceThreshold
#selector = VarianceThreshold()
#df = selector.fit_transform(fact_data)
#print(df.shape)
#df

In [28]:
# 방법 2
# 분산 구하기
df_var = fact_data.var()    # 시리즈
# 확인
print(df_var.describe())

# 분산이 0에 수렴하는 컬럼 확인
## 10e-10 이하면 0에 수렴하는 값으로 간주함
print(df_var[df_var.values <= 10e-10])

# 삭제
fact_data.drop(df_var[df_var.values <= 10e-10].index, axis=1, inplace=True)

# 컬럼 수 확인
print(len(fact_data.columns))

count    7.320000e+02
mean     9.906385e-01
std      9.741771e-02
min      7.538541e-27
25%      1.000222e+00
50%      1.000674e+00
75%      1.000815e+00
max      1.001149e+00
dtype: float64
LEHR.DRV.MTR..M4.2.CUR.1WII40105.PV     7.538541e-27
LEHR.DRV.MTR..M9.1.CUR.1WII40114.PV     7.538541e-27
LEHR.DRV.MTR..M9.2.CUR.1WII40115.PV     7.538541e-27
LEHR.DRV.MTR..M10.1.CUR.1WII40116.PV    7.538541e-27
LEHR.DRV.MTR..M14.1.CUR.1WII40124.PV    7.538541e-27
LEHR.DRV.MTR..M17.2.CUR.1WII40131.PV    7.538541e-27
TMP.TIN..BAY.4.LEFT.1TI30203.PV         7.538541e-27
dtype: float64
725


### 피처 간 상관관계 높은 피처 제거
- 상관관계 : 피처 간의 종속된 정도
- 상관관계가 높은 피처를 제거하는 이유?
    - 두 피처 간의 상관관계가 높다는 것은, 하나의 피처 값이 다른 피처의 값에 큰 영향을 주고있음을 의미함
    - 두 피처는 동일한 원인에 기인하여 변하는 것으로 추측할 수 있음
    - 이를 제거하지 않고 두면 사실상 같은 의미인 데이터가 모델링에 여러 번 반영됨
    - 사실상 종속관계에 있는 피처들이 모델링에 크게 기여하는 것과 같음
    - 모델링에 영향을 미치는 원인들이 모두 비슷한 중요도로 반영되게 하려면 종속성이 낮은 피처들만을 이용하여 모델을 만드는 것이 타당함
- 여기서는 피처 간 상관계수의 절대값이 0.9 이상인 경우를 종속된 것으로 봄


In [29]:
# 모든 컬럼에 대해 상관계수를 구하고 상관계수가 0.9 이상인 컬럼 중 1개만 남기고 모두 제거

# 모든 컬럼에 대해 상관계수의 절대값 구하기
corr_arr = abs(fact_data.corr())

# 상관계수가 0.9 이상인 컬럼 중 1개만 남기고 모두 제거
del_list = []
for i, col in enumerate(corr_arr.columns):
    for j in range(len(corr_arr.index)):
        if i == j:
            continue
        elif corr_arr.index[i] in del_list:           
            break
        # print(i, j, corr_arr.index[j], np.abs(corr_arr.iloc[j,i]))
        if np.abs(corr_arr.iloc[j,i]) >= 0.9:
            del_list.append(corr_arr.index[j])
            # print(corr_arr.index[j], '-->', corr_arr.index[i])
# 확인
len(del_list)

630

In [30]:
# del_list에 있는 컬럼은 제외
total_col = fact_data.columns.tolist()
col_remain = []
for c in total_col:
    if c not in del_list:
        col_remain.append(c)
len(col_remain)

324

In [31]:
# 저장
fact_data = fact_data[col_remain]
fact_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8145 entries, 2016-01-01 07 to 2016-12-31 23
Columns: 324 entries, LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101 to STD_GLS_L
dtypes: float64(324)
memory usage: 20.2+ MB


### (안함) VIF > 30 이상의 피처 제거

In [32]:
# 화이팅

### 전처리 수행 후 파일 저장

In [52]:
# 전처리 수행한 파일 저장하기
fact_data.to_csv('/content/drive/MyDrive/나무플래닛/3. 산출물/fact_data.csv')

In [54]:
# 확인
fact_data= pd.read_csv('/content/drive/MyDrive/나무플래닛/3. 산출물/fact_data.csv')
fact_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8145 entries, 0 to 8144
Columns: 325 entries, dat...date.name. to STD_GLS_L
dtypes: float64(324), object(1)
memory usage: 20.2+ MB
